In [78]:
import numpy as np
import matplotlib.pyplot as plt

# Q1

In [79]:
def decomp1d(n, num_blocks, sub_mat_index):
    """_summary_

    Args:
        n (int): matrix lenght
        num_blocks (int): number of blocks to split into
        sub_mat_index (int): index of sub matrix row

    Returns:
        _type_: _description_
    """
    remainder = n % num_blocks
    base = n // num_blocks

    if (sub_mat_index < remainder):
        s = sub_mat_index * (base+1)
        e = s + base+1
        return s, e  # start, end
    else:
        s = (remainder * (base+1)) + (max(sub_mat_index-remainder, 0) * base)
        e = s + base
        return s, e 
   
def decomp_matrix(A, block_rows, block_cols):
    block_A = [[0]*block_cols for i in range(block_rows)]
    m, n = len(A), len(A[0])
    
    for i in range(block_rows):
        s1, e1 = decomp1d(m, block_rows, i)
        for j in range(block_cols):
            s2, e2 = decomp1d(n, block_cols, j)
            sub_block_rows = e1 - s1
            sub_block_cols = e2 - s2
            block_A[i][j] = np.zeros((sub_block_rows, sub_block_cols))
            for a in range(sub_block_rows):
                for b in range(sub_block_cols):
                    block_A[i][j][a][b] = A[s1 + a][s2 + b]

    return block_A

def comp_matrix(m, n, block_mat):
    B = np.zeros((m, n))
    block_rows = len(block_mat)
    block_cols = len(block_mat[0])
    
    
    row_step = 0
    col_step = 0
    sub_rows = 0
    sub_cols = 0
    
    for i in range(block_rows):
        for j in range(block_cols):
            block = np.array(block_mat[i][j])
            
            sub_rows, sub_cols = len(block), len(block[0])
            for a in range(sub_rows):
                for b in range(sub_cols):
                    B[row_step + a][col_step + b] = block[a][b]
                    
            col_step += sub_cols
        row_step += sub_rows
        col_step = 0
            
    return B

def mgs(A):
    """Calculates QR decomposition of a matrix A using the modified Gram Schmidt algorithm.

    Args:
        A (numpy array): numpy matrix

    Returns:
        tuple: the pair Q, R
    """
    #m, n = A.shape
    m, n = len(A), len(A[0])
    print(f"m, n = {m}, {n}")
    
    a = [A[:, i] for i in range(n)]
    
    Q = np.zeros((m, n))
    q = [Q[:, i] for i in range(n)]
    
    V = np.zeros((m, n))
    v = [V[:, i] for i in range(n)]
    
    R = np.zeros((n, n))
    r = [R[:, i] for i in range(n)]
    
    for i in range(n):
        v[i] = a[i]
    for i in range(n):
        r[i][i] = np.linalg.norm(v[i])
        q[i] = v[i] / r[i][i]
        for j in range(i, n):
            r[i][j] = q[i].T @ v[j]
            v[j] = v[j] - (r[i][j]*q[i])
    
    Q = np.reshape(np.array(q), (m,n))
    R = np.reshape(np.array(r), (n,n))
    
    return Q, R

In [158]:
def TSQR(A, max_block_rows):
    m, n = len(A), len(A[0])
    Q = np.eye(m)
    R = decomp_matrix(A, max_block_rows, 1)
    
    rows_num = max_block_rows
    while rows_num > 1:
        
        Q_temp = [[None]*rows_num for _ in range(rows_num)]
        R_temp = [None]*rows_num
        
        for i in range(rows_num):
            Qi, Ri = np.linalg.qr(R[i][0] , mode="complete")
            
            R_temp[i] = Ri
            Q_temp[i][i] = Qi
            
        for i in range(rows_num):
            for j in range(rows_num):
                if i!=j:
                    m0, n0 = len(Q_temp[i][i]), len(Q_temp[j][j])
                    Q_temp[i][j] = np.zeros((m0, n0))
           
        Q = Q @ comp_matrix(m, m, Q_temp)
        
        rows_num = rows_num // 2
        R = [None]*rows_num
        for i in range(rows_num):
            R1 = R_temp[i*2]
            R2 = R_temp[i*2 + 1]
            m0 = len(R1) + len(R2)
            n0 = n
            
            R[i] = [comp_matrix(m0, n0, [[R1], [R2]])]
        
    Q_temp, R_temp = np.linalg.qr(R[0][0])
    Q = Q @ Q_temp
    R = R_temp
    
    return Q, R

In [218]:
n = np.random.randint(low=10, high=100)
m = n * np.random.randint(low=2, high=10)
A = np.random.rand(m, n)

print(f"A.shape={A.shape}")
npQ, npR = np.linalg.qr(A)
Q, R = TSQR(A, 4)

print(f"MSE(Q - npQ) = {np.mean(np.square(Q - npQ))}")
print(f"MSE(R - npR) = {np.mean(np.square(R - npR))}")

A.shape=(292, 73)
MSE(Q - npQ) = 0.00300243948207919
MSE(R - npR) = 0.7324146865530512
